In [1]:
import os
import gc
import multiprocessing
import pandas as pd
from sklearn.linear_model import LinearRegression

PATH = '/home/kai/data/kaggle/homecredit/'
if not os.path.exists(PATH + 'inter/linear/'): os.mkdir(PATH + 'inter/linear/')
    
'done'

'done'

In [2]:
def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)


def _apply_group(args):
    func, group_id, x, y = args
    return [group_id] + func(x, y)

def apply_by_multiprocessing(func, **kwargs):
    workers = kwargs.pop('workers')
    group_list = kwargs.pop('group_list')
    pool = multiprocessing.Pool(processes=workers)
    print(len(group_list), len(group_list[0]))
    a = [(func, group_id, x, y) for group_id, x, y in group_list]
    result = pool.map(_apply_group, a)
    pool.close()
    return result

def liner_reg(x, y, **kwargs):
    lg = LinearRegression()
    lg.fit(x, y)
    
    lg1 = LinearRegression()
    x_max = max(x.iloc[:,0])
    x_min = min(x.iloc[:,0])
    lg1.fit((x-x_max)/(x_max-x_min+1), y)
    return [lg.coef_[0][0], lg1.coef_[0][0]]

def linear0(df_, gp_col, x_col_name, y_col_name, x_map=None, n_job=8):
    df = df_.copy()
    if x_map != None: df[x_col_name] = df[x_col_name].map(x_map)
    group = [(i, x[[x_col_name]], x[[y_col_name]]) for i, x in df.groupby(gp_col)]
    r = apply_by_multiprocessing(liner_reg, workers=n_job, group_list=group)
    tmpdf = pd.DataFrame(r, columns=[gp_col, x_col_name+'_lg_'+y_col_name, x_col_name+'_normallg_'+y_col_name])
    return tmpdf.astype({gp_col: df_[gp_col].dtype})

def linear(df, gp_col, x_col_name, y_col_name):
    r = []
    for i, x in df.groupby(gp_col):
        lg = LinearRegression()
        lg1 = LinearRegression()
        
        lg.fit(x[[x_col_name]], x[[y_col_name]])
        x_max = max(x[x_col_name])
        x_min = min(x[x_col_name])
        lg1.fit((x[[x_col_name]]-x_max)/(x_max-x_min+1), x[[y_col_name]])
        r.append([i, lg.coef_[0][0], lg1.coef_[0][0]])
    tmpdf = pd.DataFrame(r, columns=[gp_col, x_col_name+'_lg_'+y_col_name, x_col_name+'_normallg_'+y_col_name])
    return tmpdf.astype({gp_col: df[gp_col].dtype})

# installments payments

In [3]:
inst = pd.read_csv(PATH + 'installments_payments.csv')

In [4]:
%%time

name1 = minus_name('DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT')
inst[name1] = minus(inst, 'DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT')
inst.at[inst[name1] < 0, name1] = 0
name2 = minus_name('AMT_INSTALMENT', 'AMT_PAYMENT')
inst[name2] = minus(inst, 'AMT_INSTALMENT', 'AMT_PAYMENT')
inst.at[inst[name2] < 0, name2] = 0
name3 = ratio_name(name2, name1)
inst[name3] = ratio(inst, name2, name1)
print(inst.shape)

r = None
for x in [name1, name2, name3]:
    print(x)
    if r is None: r = linear(inst.fillna(0), 'SK_ID_CURR', 'DAYS_INSTALMENT', x)
    else: r = r.merge(linear(inst.fillna(0), 'SK_ID_CURR', 'DAYS_INSTALMENT', x), on='SK_ID_CURR', how='left')
    print(r.shape)

columns = []
for x in r.columns:
    tmp = 'install_' + x if x != 'SK_ID_CURR' else x
    columns.append(tmp)
r.columns = columns
'done'

(13605401, 11)
DAYS_ENTRY_PAYMENT_minus_DAYS_INSTALMENT
(339587, 3)
AMT_INSTALMENT_minus_AMT_PAYMENT
(339587, 5)
AMT_INSTALMENT_minus_AMT_PAYMENT_divide_DAYS_ENTRY_PAYMENT_minus_DAYS_INSTALMENT
(339587, 7)
CPU times: user 17h 14min 54s, sys: 56min 2s, total: 18h 10min 56s
Wall time: 1h 3min 28s


In [5]:
r.to_pickle(PATH + 'inter/linear/linearinstal2curr.pkl')

# bureau

In [6]:
bureau = pd.read_csv(PATH + 'bureau.csv')
bb = pd.read_csv(PATH + 'bureau_balance.csv')

In [ ]:
tmp_df = pd.DataFrame(bureau[['SK_ID_CURR', 'SK_ID_BUREAU']])
tmp_df1 = bb[(bb['STATUS']!='C') & (bb['STATUS']!='X')]
tmp_df2 = bb[bb['STATUS'] != 'X']
x_map = {'C':0, '0':0, '1':1, '2':2, '3':3, '4':4, '5':5}

tmp_df1['STATUS'] = tmp_df1['STATUS'].map(x_map)
tmp_df2['STATUS'] = tmp_df2['STATUS'].map(x_map)
print('map done')

tmp_df = tmp_df.merge(linear(tmp_df1, 'SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'), on='SK_ID_BUREAU', how='left')
tmp_df = tmp_df.merge(linear(tmp_df2, 'SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'), on='SK_ID_BUREAU', how='left')

r = None
agg_list = ['mean', 'median', 'max', 'min']
for x in tmp_df.columns:
    if x not in ['SK_ID_BUREAU', 'SK_ID_CURR']:
        if r is None: r = numerical(tmp_df, 'SK_ID_CURR', x, agg_list)
        else: r = r.merge(numerical(tmp_df, 'SK_ID_CURR', x, agg_list), on='SK_ID_CURR', how='left')

columns = []
for x in r.columns:
    tmp = 'bureau_' + x if x != 'SK_ID_CURR' else x
    columns.append(tmp)
r.columns = columns
'done'

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


map done


In [ ]:
r.to_pickle(PATH + 'inter/linearbureau2curr.pkl')

# pos cash balance

In [ ]:
pos = pd.read_csv(PATH + 'POS_CASH_balance.csv')

In [ ]:
r = None
for x in ['SK_DPD']:
    print(x)
    if r is None: r = linear(pos.fillna(0), 'SK_ID_CURR', 'MONTHS_BALANCE', x)
    else: r = r.merge(linear(pos.fillna(0), 'SK_ID_CURR', 'MONTHS_BALANCE', x), on='SK_ID_CURR', how='left')
    print(r.shape)

columns = []
for x in r.columns:
    tmp = 'poscash_' + x if x != 'SK_ID_CURR' else x
    columns.append(tmp)
r.columns = columns
'done'

In [ ]:
r.to_pickle(PATH + 'inter/linearposcash2curr.pkl')

# credit card balance

In [ ]:
credit = pd.read_csv(PATH + 'credit_card_balance.csv')

In [ ]:
r = None
name1 = minus_name('AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL')
name2 = minus_name('AMT_BALANCE', 'AMT_PAYMENT_TOTAL_CURRENT')
credit[name1] = minus(credit, 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL')
credit[name2] = minus(credit, 'AMT_BALANCE', 'AMT_PAYMENT_TOTAL_CURRENT')

credit.at[credit[name1]<0, name1] = 0
credit.at[credit[name2]<0, name2] = 0

for x in ['SK_DPD', 'AMT_BALANCE', 'AMT_DROWINGS_CURRENT', 'AMT_TOTAL_REVEIVABLE', 'SK_DPD', name1, name2]:
    print(x)
    if r is None: r = linear(credit.fillna(0), 'SK_ID_CURR', 'MONTHS_BALANCE', x)
    else: r = r.merge(linear(credit.fillna(0), 'SK_ID_CURR', 'MONTHS_BALANCE', x), on='SK_ID_CURR', how='left')
    print(r.shape)

columns = []
for x in r.columns:
    tmp = 'poscash_' + x if x != 'SK_ID_CURR' else x
    columns.append(tmp)
r.columns = columns
'done'

In [ ]:
r.to_pickle(PATH + 'inter/linearcredit2curr.pkl')